<a href="https://colab.research.google.com/github/iannashon/cse6250/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
# pip install tensorflow_decision_forests

In [20]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import string
import tensorflow as tf
from gensim.models.keyedvectors import KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_decision_forests as tfdf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [21]:
dataset = pd.read_csv("drive/MyDrive/mimicdb/dataset_05.csv")
# dataset.count()

In [22]:
# dataset.head()
def check_imbalance(dataset):
  xx = dataset["isReadmission"].value_counts().reset_index()
  sns.barplot(x="index", y="isReadmission", data=xx, palette="cividis")

In [23]:
# X_train, X_test, y_train, y_test = train_test_split(dataset["TEXT"], dataset["TEXT_LN"], test_size=0.1, random_state=42)

In [24]:
#  check_imbalance(X_test)

In [25]:
# msk = np.random.rand(len(dataset)) < 0.3
# sample_df = dataset[msk]
# print(len(sample_df))
# dataset= sample_df

In [26]:
def remove_numbers_and_special_character(text):
    text_cln = re.sub('[^A-Za-z]+', ' ', str(text))
    return text_cln
  

def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()


def split_dataset(train_data):
  X_train, X_valid, y_train, y_valid = train_test_split(train_data['TEXT'].tolist(),\
                                                      train_data['isReadmission'].tolist(),\
                                                      test_size=0.1,\
                                                      random_state=42)
  return X_train, X_valid, y_train, y_valid

def label_tx(y_train):
  le = LabelEncoder()
  train_labels = le.fit_transform(y_train)
  # train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))
  return train_labels

In [27]:
dataset['TEXT'] = dataset['TEXT'].apply(remove_numbers_and_special_character)
dataset['TEXT'] = dataset['TEXT'].apply(clean_text)
print(len(dataset))

14040


In [28]:
X_train, X_test, y_train, y_test = split_dataset(dataset)

In [29]:
ytrain= label_tx(y_train)
ytest= label_tx(y_test)

In [30]:
tfidf_vectorizer = TfidfVectorizer() 
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

In [31]:
idf = tfidf_vectorizer.idf_
# print(idf.shape)
word_vec_model = dict(zip(tfidf_vectorizer.get_feature_names(), idf))
# print(word_vec_model)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
def word_embedding(train, model):
  features = []
  for data in train:
    # print(data)
    text_splt = data.split()
    txt_arry = []
    for text in text_splt:
      weight = model.get(text)
      # weight_arry = []
      if weight is not None:
        # weight_arry.append(weight)
        txt_arry.append(weight)
    
    txt_arry_np = np.asarray(txt_arry).astype(np.float32)
    features.append(txt_arry_np)

  return np.array(features)

def x_padding(train):
  inputs = pad_sequences(train, padding='post', maxlen=1000, dtype='float32')
  # inputs = inputs.astype(np.float32)
  return inputs

# print(X_train[0])
X_train = word_embedding(X_train, word_vec_model)
X_train = x_padding(X_train)

X_test = word_embedding(X_test, word_vec_model)
X_test = x_padding(X_test)

# print(xtrain[0][0])

<ipython-input-32-451ba355e765>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(features)


In [33]:
classifier = RandomForestClassifier()
classifier.fit(X_train,ytrain)

RandomForestClassifier()

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.75      1.00      0.85      1047
        True       0.00      0.00      0.00       357

    accuracy                           0.75      1404
   macro avg       0.37      0.50      0.43      1404
weighted avg       0.56      0.75      0.64      1404



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
model = KNeighborsClassifier(n_neighbors=2)

# Train the model using the training sets
model.fit(X_train,ytrain)

#Predict Output
predicted= model.predict(X_test) # 0:Overcast, 2:Mild
# print(predicted)

In [37]:
print(classification_report(y_test,predicted))

              precision    recall  f1-score   support

       False       0.75      0.82      0.78      1047
        True       0.27      0.19      0.22       357

    accuracy                           0.66      1404
   macro avg       0.51      0.51      0.50      1404
weighted avg       0.63      0.66      0.64      1404

